In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [2]:
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": "C:/Users/gualas/Desktop/prueba",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

In [8]:
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://github.com/elgualas?tab=repositories")

boton_repo=driver.find_element(By.XPATH,"//*[@id='user-repositories-list']/ul/li[1]/div[1]/div[1]/h3/a")
boton_repo.click()

"""boton_code=driver.find_element(By.XPATH,"//*[@id='repo-content-pjax-container']/div/div/div[3]/div[1]/div[1]/span[1]/get-repo/details/summary")
boton_code.click()"""#lo que no funciona




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='repo-content-pjax-container']/div/div/div[3]/div[1]/div[1]/span[1]/get-repo/details/summary"}
  (Session info: chrome=110.0.5481.100)
Stacktrace:
Backtrace:
	(No symbol) [0x011A37D3]
	(No symbol) [0x01138B81]
	(No symbol) [0x0103B36D]
	(No symbol) [0x0106D382]
	(No symbol) [0x0106D4BB]
	(No symbol) [0x010A3302]
	(No symbol) [0x0108B464]
	(No symbol) [0x010A1215]
	(No symbol) [0x0108B216]
	(No symbol) [0x01060D97]
	(No symbol) [0x0106253D]
	GetHandleVerifier [0x0141ABF2+2510930]
	GetHandleVerifier [0x01448EC1+2700065]
	GetHandleVerifier [0x0144C86C+2714828]
	GetHandleVerifier [0x01253480+645344]
	(No symbol) [0x01140FD2]
	(No symbol) [0x01146C68]
	(No symbol) [0x01146D4B]
	(No symbol) [0x01150D6B]
	BaseThreadInitThunk [0x763C00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77E67BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77E67B8E+238]


In [1]:
print('prueba1')

prueba1
